In [ ]:
import pandas as pd
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch
import numpy as np
from constant import AFFECTNETPATH

In [2]:
class ImageDataset(Dataset):
    def __init__(self, csv_file,base_path,transform=None):
        self.fields = ['subDirectory_filePath', 'expression','valence','arousal']
        self._table = pd.read_csv(csv_file,usecols=self.fields)
        
        self._table=self._table[self._table['expression'] <9]

        self._table=self._table.reset_index(drop=True)

        self._base_path=base_path
   



        self.transform=transforms.Compose([
                     transforms.Resize((64,64)),
                     transforms.ToTensor(),   
                     transforms.Normalize((0.5,0.5,0.5 ), (0.5, 0.5,0.5))])

    def __len__(self):
        return len(self._table)

    def __getitem__(self, idx):
        folder_name = os.path.join(self._base_path,self._table.subDirectory_filePath[idx])
        img=Image.open(folder_name)
        img=self.transform(img)
        valence = torch.from_numpy(np.array(self._table.valence[idx]))
        arousal = torch.from_numpy(np.array(self._table.arousal[idx]))
        return img,valence.float(),arousal.float()
            
        

In [3]:
Affect_Train=ImageDataset('training.csv',AFFECTNETPATH)
Affect_Valid=ImageDataset('validation.csv',AFFECTNETPATH)

In [4]:
len(Affect_Train)

320740

In [5]:
len(Affect_Valid)

4500

In [ ]:
try :
    for i,file in enumerate(Affect_Valid):
        if i%10000==0:
            print(file[1])
except FileNotFoundError:
    
    print("Error Occurs in:", i)
    pass 
    

In [ ]:
try :
    for i,file in enumerate(Affect_Train):
        if i%10000==0:
            print(file[1])
except FileNotFoundError:
    
    print("Error Occurs in:", i)
    pass 
    

In [ ]:
Affect_Train[243557][0]

In [6]:
train_dataloader = DataLoader(Affect_Train, batch_size=256
                       , num_workers=0,shuffle=True)

valid_dataloader = DataLoader(Affect_Valid, batch_size=256
                       , num_workers=0)    

In [7]:
len(train_dataloader)

1253

In [8]:
Affect_Train[0][0].shape

torch.Size([3, 64, 64])

In [9]:
from module import face_feature

In [14]:
model=face_feature.Face_Feature()
model=model.to(device)

In [15]:
model(Affect_Train[0][0].unsqueeze(0).to(device))

(tensor([[0.3269]], grad_fn=<TanhBackward>),
 tensor([[0.3715]], grad_fn=<TanhBackward>))

In [13]:
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0002, betas=(0.5, 0.999))

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [ ]:
train_loss=[]
num_epochs=50

model.train()
for epochs in range(0,num_epochs):
    model.train()
    
    print("Training Epoch: ", epochs+1,"\n")
    for i_batch, (img,valence,arousal) in enumerate(train_dataloader):
        
        batch_size=img.size(0)
        optimizer.zero_grad()
        img=img.to(device)
        val_output,arou_output=model(img)
        loss1=loss_func(val_output,valence.to(device))
        loss2=loss_func(arou_output,valence.to(device))
        loss=loss1+loss2
        loss.backward()
        optimizer.step()
        if i_batch%200==0:
            print("Batch: ",i_batch+1,"/",len(train_dataloader))
            print("MSE loss: ", loss.item())
            train_loss.append(loss.item())
    model.eval()
    avg_loss=1e6
    temp_loss=0
    
    for i_batch, (img,valence,arousal) in enumerate(train_dataloader):
        
        batch_size=img.size(0)

        img=img.to(device)
        val_output,arou_output=model(sample_batched)
        loss1=loss_func(val_output,valence.to(device))
        loss2=loss_func(arou_output,valence.to(device))
        loss=loss1+loss2
        temploss+=loss.item()
    print("ValidationLoss: ",temp_loss/len(valid_dataloader))
    if temp_loss<avg_loss:
        avg_loss=temp_loss
        torch.save(model,"face_feature.h")
    
        

Training Epoch:  1 



/opt/conda/envs/torch/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Batch:  1 / 1253
MSE loss:  0.5873039960861206
Batch:  201 / 1253
MSE loss:  0.6043837070465088


In [ ]:
avg_loss=1e6

In [ ]:
avg_loss